<a href="https://colab.research.google.com/github/handsome1201/Failure/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bar graph & Circle graph

In [1]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install pyyaml==5.1
# !pip install torch== 1.11.0 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q pytesseract
!pip install transformers
!pip install datasets
!sudo apt-get install tesseract-ocr
import cv2
import numpy as np
from datasets import load_dataset
import pytesseract
from PIL import Image
!pip install easyocr
import easyocr
import xml.etree.ElementTree as ET
!pip install extcolors
import extcolors
import os
import re

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-7k5km6u0
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-7k5km6u0
  Resolved https://github.com/facebookresearch/detectron2.git to commit ff53992b1985b63bd3262b5a36167098e3dada02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.6-cp310-cp310-linux_x86_64.whl size=7803906 sha256=610ff37287d1305d371d8888e8f659c

### 그래프 종류 판별

In [2]:
def detect_graph_orientation(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=250, maxLineGap=15)

    if lines is not None:
        vertical_lines = []
        horizontal_lines = []

        for line in lines:
            x1, y1, x2, y2 = line[0]
            if abs(x1 - x2) < 5:  # Check if the line is vertical
                vertical_lines.append(line)
            elif abs(y1 - y2) < 5:  # Check if the line is horizontal
                horizontal_lines.append(line)

        if len(vertical_lines) > len(horizontal_lines):
            return 1  # "Vertical"
        elif len(horizontal_lines) > len(vertical_lines):
            return 2  # "Horizontal"
        else:
            return 0

In [3]:
def wrong_detect_delete(data):
  count=0
  number=re.compile('[\d]')
  out_str=''
  for j in data :
      j = list(j)
      for alphabet in j:
          if number.match(alphabet)!=None:
              out_str+=alphabet
          elif alphabet==',' or alphabet=='.':
              out_str+=alphabet

  if data[-1]!="%" and len(out_str)>5 and (out_str[len(out_str)-2:len(out_str)] =='96') | (out_str[len(out_str)-2:len(out_str)] =='95') | (out_str[len(out_str)-2:len(out_str)] =='36'):

      out_str = list(out_str)

      out_str[len(out_str)-2:len(out_str)] = ''
      out_str=''.join(out_str)
  return out_str

In [4]:
def Vertical(image_path):

  reader = easyocr.Reader(['en', 'ko'], gpu=True)

  result = reader.readtext(image_path)

  img = Image.open(image_path)
  img_array = np.array(img)
  gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
  result = reader.readtext(gray)
  for r in result:
      x, y = min(list(_[0] for _ in r[0])), min(list(_[1] for _ in r[0]))
      text_data = r[1]
  x_values = []
  y_values = []
  grouped_values = {}
  grouped_values_list=[]

  for r in result:
      x, y = min(list(_[0] for _ in r[0])), min(list(_[1] for _ in r[0]))
      x_values.append(x)
      y_values.append(y)

  grouped_indices = []  # 묶인 값들의 인덱스를 저장할 리스트

  for i, x in enumerate(x_values):
    if i in grouped_indices:  # 이미 묶인 값인 경우 건너뜁니다
        continue

    group = [result[i][1]]  # 현재 값과 묶일 값들을 저장할 리스트
    group_indices = [i]  # 현재 값과 묶일 값들의 인덱스를 저장할 리스트

    # 값의 차이가 25 이하인 값을 찾아서 묶어줍니다
    for j in range(i+1, len(x_values)):
        if abs(x_values[j] - x) <= 25:
            group.append(result[j][1])
            group_indices.append(j)


    if len(group) > 1:  # 묶인 값들이 2개 이상인 경우에만 처리합니다
        for value in group[1:]:
            grouped_values[value] = group[0]
            grouped_indices.extend(group_indices[1:])  # 묶인 값들의 인덱스를 추가합니다
        grouped_values_list.append(group)  # group 값을 grouped_values_list에 추가합니다

  root = ET.Element("root")

  for key, value in grouped_values.items():
      element = ET.SubElement(root, "item")
      element.set("key", key)
      element.text = detect_graph_orientation(value)

  tree = ET.ElementTree(root)
  tree.write("data2.xml", encoding="utf-8")

In [5]:
def Horizontal(image_path):

  reader = easyocr.Reader(['en', 'ko'], gpu=True)

  result = reader.readtext(image_path)

  img = Image.open(image_path)
  img_array = np.array(img)
  gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
  result = reader.readtext(gray)
  for r in result:
      x, y = min(list(_[0] for _ in r[0])), min(list(_[1] for _ in r[0]))
      text_data = r[1]
  x_values = []
  y_values = []
  grouped_values = {}
  grouped_values_list = []

  for r in result:
      x, y = min(list(_[0] for _ in r[0])), min(list(_[1] for _ in r[0]))
      x_values.append(x)
      y_values.append(y)

  grouped_indices = []  # 묶인 값들의 인덱스를 저장할 리스트

  for i, y in enumerate(y_values):
    if i in grouped_indices:  # 이미 묶인 값인 경우 건너뜁니다
        continue

    group = [result[i][1]]  # 현재 값과 묶일 값들을 저장할 리스트
    group_indices = [i]  # 현재 값과 묶일 값들의 인덱스를 저장할 리스트

    # 값의 차이가 25 이하인 값을 찾아서 묶어줍니다
    for j in range(i+1, len(y_values)):
        if abs(y_values[j] - y) <= 25:
            group.append(result[j][1])
            group_indices.append(j)

    if len(group) > 1:  # 묶인 값들이 2개 이상인 경우에만 처리합니다
        for value in group[1:]:
            grouped_values[value] = group[0]
            grouped_indices.extend(group_indices[1:])  # 묶인 값들의 인덱스를 추가합니다
        grouped_values_list.append(group)  # group 값을 grouped_values_list에 추가합니다

  root = ET.Element("root")

  for value,key in grouped_values.items():
      element = ET.SubElement(root, "item")
      element.set("key", key)
      element.text = wrong_detect_delete(value)

  tree = ET.ElementTree(root)
  tree.write("data2.xml", encoding="utf-8")

In [6]:
def circle(image_path):
  def compare_rgb_tuple(rgb1, rgb2):  # 두 rgb 값들이 유사한지를 판단해 boolean으로 반환하는 함수
      root = ET.Element("root")
      r_diff_abs = abs(rgb1[0] - rgb2[0])
      g_diff_abs = abs(rgb1[1] - rgb2[1])
      b_diff_abs = abs(rgb1[2] - rgb2[2])
      tot_abs = r_diff_abs + g_diff_abs + b_diff_abs
      if r_diff_abs<30 and g_diff_abs<30 and b_diff_abs<30 and tot_abs<70 :
          return True
      return False

  img = Image.open(image_path)	# 이미지 열기
  rgb_img = img.convert("RGB")

  reader = easyocr.Reader(['ko','en'], gpu=True)	# ocr로 이미지 읽기
  result1 = reader.readtext(image_path)

  num = len(result1)
  text_list=[]	# ocr로 읽은 text list
  color_list=[]	# text가 위치한 곳의 color list

  for i in range(num):	# text가 위치한 곳의 rgb 값을 읽어옴.
      min_x=result1[i][0][0][0]
      min_y=result1[i][0][0][1]
      width=0
      height=0
      text_list.append(result1[i][1])
      for j in range(3):
          j=j+1
          if min_x>=result1[i][0][j][0]:
              min_x=result1[i][0][j][0]
          else :
              width = result1[i][0][j][0]-min_x
          if min_y>=result1[i][0][j][1]:
              min_y=result1[i][0][j][1]
          else :
              height = result1[i][0][j][1]-min_y
      crop_img = rgb_img.crop((min_x, min_y, min_x+width, min_y+height))
      crop_img.save('tb_img_ext'+str(i)+'.png', 'png')
      saved_croped_img = Image.open('tb_img_ext'+str(i)+'.png')
      colors, _ = extcolors.extract_from_image(saved_croped_img, limit=1)
      for c in colors:
          color_list.append(c[0])
      os.remove('tb_img_ext'+str(i)+'.png')


  rgb_group=[]
  group=[]
  arr_count=0
  filled_group=0
  for rgb_val in color_list:	# text의 rgb 값이 겹치는 것끼리 group화
      if len(rgb_group)==0 :
          rgb_group.append(rgb_val)
          group.append([])
          group[filled_group].append(text_list[arr_count])
          filled_group+=1
      else :
          is_old=0
          group_count=0
          for selected_rgb in rgb_group:
              if compare_rgb_tuple(selected_rgb, rgb_val):
                  group[group_count].append(text_list[arr_count])
                  is_old=1
                  break
              group_count+=1
          if is_old==0:
              rgb_group.append(rgb_val)
              group.append([])
              group[filled_group].append(text_list[arr_count])
              filled_group+=1
      arr_count+=1

  merged_group=[]
  count=0
  for i in group :		# group화된 text들을 항목과 항목값을 나누어 저장
      out_str=''
      merged_group.append([[],[]])
      for j in i:
          if (j[len(j)-2:len(j)] =='96') | (j[len(j)-2:len(j)] =='95') | (j[len(j)-2:len(j)] =='36'):
              j = list(j)
              j[len(j)-2:len(j)] = '%'
              ''.join(j)
              out_percent=''
              for k in j:
                  out_percent+=str(k)
              merged_group[count][1].append(out_percent)
              check=1
          else :
              out_str+=j
      merged_group[count][0].append(out_str)
      count+=1

  grouped_values = {}
  for i in merged_group:	# list를 dict로 변환
      grouped_values[str(i[0])]=str(i[1])

  root = ET.Element("root")

  for key, value in grouped_values.items():
      element = ET.SubElement(root, "item")
      element.set("key", key)
      element.text = wrong_detect_delete(value)

  tree = ET.ElementTree(root)
  tree.write("data2.xml", encoding="utf-8")

### input image

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
image_path = '/content/drive/MyDrive/자료/수평 한글.png'
img = cv2.imread(image_path)
orientation = detect_graph_orientation(img)

error: ignored

In [ ]:
if orientation == 1: #수직 그래프
  Vertical(image_path)

elif orientation == 2: #수평 그래프
  Horizontal(image_path)

else: #원 그래프
  circle(image_path)

In [ ]:
tree = ET.ElementTree(file="data2.xml")
root = tree.getroot()
xml_content = ET.tostring(root, encoding="utf-8").decode("utf-8")
print(xml_content)